# Watershed Delineation Methods

Run delineations for both the "series" and "parallel" approaches.

In [25]:
from pathlib import Path
import json
from statistics import mean
import math
from typing import List
from pprint import PrettyPrinter

from arcpy import FeatureSet, RecordSet, Raster, Describe, CreateUniqueName, EnvManager, SpatialReference
from arcpy.management import (
    MakeFeatureLayer, 
    SelectLayerByAttribute, 
    GetCount, 
    MakeTableView, 
    BuildRasterAttributeTable,
    Dissolve,
    AddFields,
    CreateFeatureclass
)
from arcpy.analysis import Buffer
from arcpy.sa import Watershed, FlowLength, ZonalStatisticsAsTable, SetNull, IsNull
from arcpy.da import SearchCursor, InsertCursor
from arcpy.conversion import RasterToPolygon, JSONToFeatures

from dataclasses import dataclass, field, asdict
from marshmallow import Schema, fields, EXCLUDE, pre_load
from marshmallow_dataclass import class_schema

from tqdm.notebook import tqdm
import petl as etl

# import arcgis

In [29]:
pp = PrettyPrinter(indent=2).pprint

In [2]:
from arcpy import env
env.overwriteOutput = True

In [3]:
wd = Path(r"D:\Dropbox (CivicMapper)\Projects\202004-02 Cornell Modeling\3 - Production\tool outputs")
test_results_dir = wd / "c19 tests" / "test_results"

rainfall_rasters_config = Path(r"D:\Dropbox (CivicMapper)\Projects\202004-02 Cornell Modeling\3 - Production\tool outputs\c19 tests\noaa_rainfall\noaa_rainfall_rasters_config_ne.json")

inlets = wd / "c19 tests"  / "c19_snapped_points.shp"
dem = wd / "c19 tests" / "dem_clip_simple.tif"
fd = wd / "c19 tests" / "dem_clip_simple_flowdir.tif"
cn = wd / "c19 tests" / "curveno.tif"
slope = wd / "c19 tests" / "dem_slope.tif"

pour_point_field = "NAACC_ID"
out_catchment_polygons_simplify = False

## Models:

In [40]:
@dataclass
class Rainfall:
    """storm frequency interval with rainfall values and rainfall-dependent 
    analytical results from the calculators
    """

    freq: int = None
    dur: str = None
    value: float = None
        
RainfallSchema = class_schema(Rainfall)

In [41]:
@dataclass
class Shed:
    """Characteristics of a single point's contributing area
    """
    # unique id field, derived from the outlet point; the value from the
    # "pour_point_field". For NAACC-based culvert modeling, this is the
    # NAACC Naacc_Culvert_Id field
    uid: str = None
    
    # a group id field. non-unique ID field that indicates groups of related
    # outlets. Used primarily for NAACC-based culvert modeling, this is the
    # NAACC Survey_Id field
    group_id: str = None

    # characteristics used for calculating peak flow
    area_sqkm: float = None# <area of inlet's catchment in square km>
    avg_slope_pct: float = None # <average slope of DEM in catchment>
    avg_cn: float = None # <average curve number in the catchment>
    max_fl: float = None # <maximum flow length in the catchment>
    rainfall: List[Rainfall] = None # average rainfall in the catchment, for storm events

    # geometries
    inlet_geom: dict = None
    shed_geom: dict = None
    
    # for recording the location of intermediate geospatial output files
    shed_polygon_filepath: str = None
    shed_raster_filepath: str = None
        
ShedSchema = class_schema(Shed)

In [42]:
@dataclass
class Point:
    """Basic model for points used as source delineations for peak-flow-calcs;
    minimal attributes required.
    """

    # unique id field, derived from the outlet point; the value from the
    # "pour_point_field". For NAACC-based culvert modeling, this is the
    # NAACC Naacc_Culvert_Id field
    uid: str

    lat: float = None
    lng: float = None
    spatial_ref_code: int = None

    # a group id field. non-unique ID field that indicates groups of related
    # outlets. Used primarily for NAACC-based culvert modeling, this is the
    # NAACC Survey_Id field
    group_id: str = None

    # optionally extend with NAACC & capacity attributes
    # naacc: NaaccCulvert = None
    # capacity: Capacity = None
    naacc: dict = None
    capacity: dict = None

    # optionally extend with the Shed and its characteristics
    shed: Shed = None

    # rainfall frequency-based analytical results for the point
    analytics: List[Rainfall] = field(default_factory=list)

    # flags, errors, and notes
    include: bool = True
    validation_errors: dict = field(default_factory=dict)
    notes: list = field(default_factory=list)

    # place to store the raw input
    raw: dict = None

In [7]:
def fc_to_petl_table(
    feature_class, 
    include_geom=False,
    return_featureset=True
    ):
    """Convert an Esri Feature Class to a PETL table object. Optionally,
    return the FeatureSet used to create the PETL table.

    Convert an Esri Feature Class to a PETL table object.

    :param feature_class: [description]
    :type feature_class: [type]
    :param include_geom: [description], defaults to False
    :param include_geom: bool, optional
    :param return_featureset: return Esri FeatureSet, defaults to False
    :param return_featureset: bool, optional        
    :return: tuple containing an PETL Table object and FeatureSet
    :rtype: Tuple(petl.Table, FeatureSet)
    """
    print("Reading {0} into a PETL table object".format(feature_class))
    
    feature_class = str(inlets)
    feature_set = FeatureSet(feature_class)
    # convert the FeatureSet object to a python dictionary
    fs = json.loads(feature_set.JSON)

    # describe the feature class
    described_fc = Describe(feature_class)
    # get a list of field objects
    field_objs = described_fc.fields

    # make a list of fields that doesn't include the Object ID field
    all_fields = [f for f in field_objs if f.name != described_fc.OIDFieldName]
    # make a list of field names, excluding geometry and the OID field
    attr_fields = [f.name for f in all_fields if f.type not in ['Geometry', 'Shape']]
    # then add 'geometry' to that list, since it will be present in the features in the FeatureSet
    attr_fields.append('geometry')
    
    table = etl\
        .fromdicts(fs['features'])\
        .unpackdict('attributes')\
        .cut(*attr_fields)

    if return_featureset:
        return table, feature_set
    else:
        return table, None

In [8]:
def create_geodata_from_points(
    points, 
    output_points_filepath=None,
    as_dict=True,
    default_spatial_ref_code=4326
    ):
    """from a list of Drain-It Point objects, create an ArcPy FeatureSet,
    for use in other ArcPy GP tools.

    :param points: list of drainit.models.Point objects. Only the uid and group_id are used here; extended attributes from naacc aren't used.
    :type points: List[Point]
    :param output_points_filepath: optional path to save points to file on disk, defaults to None
    :type output_points_filepath: str, optional
    :param as_dict: return the geodata as geoservices JSON as Python dictionary, defaults to True
    :type as_dict: bool, optional
    :return: geoservices JSON as Python dictionary; FeatureSet object if as_dict is False
    :rtype: dict
    """

    with EnvManager(overwriteOutput=True):

        #get the spatial ref from the first available point
        p_srs = [p.spatial_ref_code for p in points if p.spatial_ref_code is not None]
        print(p_srs)
        if len(p_srs) > 0:
            try:
                spatial_ref = SpatialReference(p_srs[0])
            except:
                spatial_ref = SpatialReference(default_spatial_ref_code)
        else:
            spatial_ref = SpatialReference(default_spatial_ref_code)
            
        print(spatial_ref)

        # Create an in_memory feature class to initially contain the points
        feature_class = CreateFeatureclass(
            out_path="in_memory", 
            out_name="temp_drainit_points", 
            geometry_type="POINT",
            spatial_reference=spatial_ref
        )

        AddFields(
            feature_class, 
            [
                #[Field Name, Field Type]
                ['uid', 'TEXT'],
                ['group_id', 'TEXT'],
            ]
        )
        
        print(points)

        # Open an insert cursor
        with InsertCursor(feature_class, ['uid', 'group_id', "SHAPE@XY"]) as cursor:
            # Iterate through list of coordinates and add to cursor
            for pt in points:
                row = [pt.uid, pt.group_id, (pt.lng, pt.lat)]
                cursor.insertRow(row)

        # Create a FeatureSet object and load in_memory feature class JSON as dict
        feature_set = FeatureSet()
        feature_set.load(feature_class)

    if output_points_filepath:
        feature_set.save(output_points_filepath)

    # return the dictionary (geoservices JSON as Python dictionary)
    if as_dict:
        return json.loads(feature_set.JSON)
    else:
        return feature_set

def extract_points_from_geodata(
    points_filepath, 
    uid_field,
    is_naacc=False,
    group_id_field=None,
    output_points_filepath=None
    ):
    """from any geodata input file, create a list of Point objects and a 
    ArcGIS FeatureSet object
    """

    # # handle inputs that are from an interactive selection in ArcMap/Pro
    if isinstance(points_filepath, FeatureSet):
        print("Reading from interactive selection")
    else:
        print('Reading from file')

    # extract feature class to a PETL table and a FeatureSet
    raw_table, feature_set = fc_to_petl_table(
        points_filepath,
        include_geom=True,
        return_featureset=True
    )

    # convert the FeatureSet to its JSON representation
    feature_set_json = json.loads(feature_set.JSON)

    # get the spatial reference code from the FeatureSet JSON
    spatial_ref_code = feature_set_json.get('spatialReference', {}).get('wkid', None)

    # if this is geodata that follows the NAACC format, we use the NAACC
    # etl function to transform the table to a list of Point objects
    # with nested NAACC and capacity calc-ready attributes where possible
    # (This is workflow for Culvert Capacity when geodata is provided 
    # instead of a CSV)
    if is_naacc:
        print('reading points and capturing NAACC attributes')
        points = etl_naacc_table(
            naacc_petl_table=raw_table,
            spatial_ref_code=spatial_ref_code
        )

    # Otherwise we transform the raw table to a list of Point objects here
    # (This is workflow for Peak-Flow)
    else:
        print('reading points')
        points = []
        for idx, r in enumerate(list(etl.dicts(raw_table))):
            point_kwargs = dict(
                uid=r[uid_field],
                lat=float(r["geometry"]['y']),
                lng=float(r["geometry"]['x']),
                spatial_ref_code=spatial_ref_code,
                include=True,
                raw=r
            )
            if group_id_field:
                point_kwargs['group_id'] = r[group_id_field],

            p = Point(**point_kwargs)
            points.append(p)

    if output_points_filepath:
        # finally, save it out
        self.msg('saving points')
        # save a copy of the points feature_set to the output location
        feature_set.save(output_points_filepath)

    # return the list of Point objects and a dict version of the FeatureSet
    return points, feature_set_json, spatial_ref_code

## Read the Points

* as a FeatureSet
* as a PETL table

In [9]:
points, fs, sr = extract_points_from_geodata(str(inlets), "NAACC_ID")

Reading from file
Reading D:\Dropbox (CivicMapper)\Projects\202004-02 Cornell Modeling\3 - Production\tool outputs\c19 tests\c19_snapped_points.shp into a PETL table object
reading points


In [10]:
points

[Point(uid=70978, lat=4690423.416399388, lng=618918.2765349224, spatial_ref_code=26918, group_id=None, naacc=None, capacity=None, shed=None, analytics=[], include=True, validation_errors={}, notes=[], raw={'OBJECTID': 1, 'pointid': 1, 'grid_code': 70978, 'Survey_ID': 73607, 'NAACC_ID': 70978, 'Lat': 42.35717, 'Long': -73.556041, 'Rd_Name': 'Raup Road', 'Culv_Mat': 'Metal', 'In_Type': 'Projecting', 'In_Shape': 'Round', 'In_A': 4.9, 'In_B': 4.8, 'HW': 2, 'Slope': 2.8, 'Length': 26, 'Out_Shape': 'Round Culvert', 'Out_A': 5.2, 'Out_B': 5.4, 'Crossing_T': 'Culvert', 'Comments': ' ', 'Flags': 1, 'Modeling_n': ' ', 'BarrierID': '96C19', 'BarrierID_': 0, 'BarrierID1': 0, 'geometry': {'x': 618918.2765349224, 'y': 4690423.416399388}}),
 Point(uid=71056, lat=4690193.416399388, lng=610968.2765349215, spatial_ref_code=26918, group_id=None, naacc=None, capacity=None, shed=None, analytics=[], include=True, validation_errors={}, notes=[], raw={'OBJECTID': 2, 'pointid': 2, 'grid_code': 71056, 'Survey_I

### Get a subset of inlets for testing purposes

In [11]:
# some_inlets = SelectLayerByAttribute(
#     in_layer_or_view=fslyr,
#     selection_type='NEW_SELECTION',
#     where_clause="Survey_ID < 67000"
# )
# inlet_count = int(GetCount(some_inlets).getOutput(0))
# print(inlet_count)

some_inlets = points[0:10]

## Read in the rainfall rasters 

from the rainfall config `json` file.

In [12]:
with open(rainfall_rasters_config) as fp:
    rainfall_rasters = json.load(fp)
    
rainfall_rasters = [
    {
        'path': Path(rainfall_rasters['root']) / r['path'],
        'freq': r['freq']
    }
    for r in rainfall_rasters['rasters'] if r['ext'] == ".asc"
]

rainfall_rasters

[{'path': WindowsPath('D:/Dropbox (CivicMapper)/Projects/202004-02 Cornell Modeling/3 - Production/tool outputs/c19 tests/noaa_rainfall/ne1yr24ha.asc'),
  'freq': 1},
 {'path': WindowsPath('D:/Dropbox (CivicMapper)/Projects/202004-02 Cornell Modeling/3 - Production/tool outputs/c19 tests/noaa_rainfall/ne2yr24ha.asc'),
  'freq': 2},
 {'path': WindowsPath('D:/Dropbox (CivicMapper)/Projects/202004-02 Cornell Modeling/3 - Production/tool outputs/c19 tests/noaa_rainfall/ne5yr24ha.asc'),
  'freq': 5},
 {'path': WindowsPath('D:/Dropbox (CivicMapper)/Projects/202004-02 Cornell Modeling/3 - Production/tool outputs/c19 tests/noaa_rainfall/ne10yr24ha.asc'),
  'freq': 10},
 {'path': WindowsPath('D:/Dropbox (CivicMapper)/Projects/202004-02 Cornell Modeling/3 - Production/tool outputs/c19 tests/noaa_rainfall/ne25yr24ha.asc'),
  'freq': 25},
 {'path': WindowsPath('D:/Dropbox (CivicMapper)/Projects/202004-02 Cornell Modeling/3 - Production/tool outputs/c19 tests/noaa_rainfall/ne50yr24ha.asc'),
  'freq

## Iterative Delineation and Analysis:

In [13]:
sheds = []

# with SearchCursor(some_inlets, [pour_point_field]) as sc:
#     for inlet in tqdm(sc, total=inlet_count):

for point in points[0:10]:
    
    print("--------------------------------")
    print("inlet", point.uid)
    
    print("deriving FeatureSet")
    feature_set_for_point = create_geodata_from_points([point], as_dict=False)    
    print(feature_set_for_point, feature_set_for_point.JSON)

#         selected_inlet = SelectLayerByAttribute(
#             in_layer_or_view=some_inlets,
#             selection_type='NEW_SELECTION',
#             where_clause="{0}= {1}".format(pour_point_field, inlet[0])
#         )

#     selected_inlet = feature_set_for_point

    # we can get a tabular look at what's in the layer like this:
#         fs = json.loads(FeatureSet(selected_inlet).JSON)
#         ft = etl.fromdicts(fs['features']).unpackdict('attributes').unpackdict('geometry')
#         etl.vis.displayall(ft)

    with EnvManager(
        snapRaster=str(fd),
        cellSize=str(fd)
    ):
        # delineate one watershed
        print('delineating')
        one_shed = Watershed(
            in_flow_direction_raster=str(fd),
            in_pour_point_data=feature_set_for_point,
            #pour_point_field=pour_point_field
        )

        #one_shed.save(str(test_results_dir / "catchment_{}.tif".format(inlet[0])))
        one_shed_path = r"in_memory/catchment_{}.tif".format(point.uid)
        one_shed.save(one_shed_path)

    print(one_shed, type(one_shed))


    # -------------------------------------------------
    # ANALYSIS


    ## ---------------------------
    # calculate area of catchment

    #ZonalGeometryAsTable(catchment_areas,"Value","output_table") # crashes like a mfer
    #cp = self.so("catchmentpolygons","timestamp","fgdb")
    cp = CreateUniqueName(
        "basin{0}".format(point.uid),
        "in_memory"
    )
    print(cp)
    #RasterToPolygon copies our ids from self.raster_field into "gridcode"
    if out_catchment_polygons_simplify:
        simplify = "SIMPLIFY"
    else:
        simplify = "NO_SIMPLIFY"

    RasterToPolygon(one_shed, cp, simplify)

    # Dissolve the converted polygons, since some of the raster zones may have corner-corner links
    #cpd = self.so("catchmentpolygonsdissolved","timestamp","fgdb")
    cpd = CreateUniqueName(
        "basin{0}_dissolved".format(point.uid),
        "in_memory"
    )

    Dissolve(
        in_features=cp,
        out_feature_class=cpd,
        dissolve_field="gridcode",
        multi_part="MULTI_PART"
    )

    # get the area for each record, and push into results object
    total_area = None
    areas = []
    with SearchCursor(cpd,["gridcode","SHAPE@AREA"]) as c:
        for r in c:
            this_id = r[0]
            this_area = r[1]# * area_conv_factor
            areas.append(this_area)

    total_area = sum(areas)
    print("area:", total_area)


    ## ---------------------------
    # calculate flow length

    fl_max = None

    with EnvManager(
        snapRaster=str(fd),
        cellSize=str(fd)
    ):

        # clip the flow direction raster to the catchment area (zone value)
        clipped_flowdir = SetNull(IsNull(one_shed), str(fd))
        # calculate flow length
        flow_len = FlowLength(clipped_flowdir, "UPSTREAM")
        # determine maximum flow length
        fl_max = flow_len.maximum
        #TODO: convert length to ? using leng_conv_factor (detected from the flow direction raster)
        #fl_max = fl_max * leng_conv_factor

    print("flow length:", fl_max)


    ## ---------------------------
    # calculate average curve number

    avg_cn = None

    table_cn_avg = CreateUniqueName(
        "basin{0}_cn_avg".format(point.uid),
        "in_memory"
    )
    with EnvManager(
        cellSizeProjectionMethod="PRESERVE_RESOLUTION",
        extent="MINOF",
        cellSize=one_shed
    ):        
        ZonalStatisticsAsTable(
            one_shed, 
            'Value', 
            Raster(str(cn)),
            table_cn_avg, 
            "DATA",
            "MEAN"
        )
        cn_stats = json.loads(RecordSet(table_cn_avg).JSON)

        if len(cn_stats['features']) > 0:
            means = [f['attributes']['MEAN'] for f in cn_stats['features']]
            avg_cn = mean(means)

        print("Average Curve Number:", avg_cn)



    ## ---------------------------
    # calculate average slope

    avg_slope = None

    table_slope_avg = CreateUniqueName(
        "basin{0}_slope_avg".format(point.uid),
        "in_memory"
    )

    with EnvManager(
        cellSizeProjectionMethod="PRESERVE_RESOLUTION",
        extent="MINOF",
        cellSize=one_shed
    ):        
        ZonalStatisticsAsTable(
            one_shed, 
            'Value', 
            Raster(str(slope)),
            table_slope_avg, 
            "DATA",
            "MEAN"
        )

        slope_stats = json.loads(RecordSet(table_slope_avg).JSON)
        if len(slope_stats['features']) > 0:
            means = [f['attributes']['MEAN'] for f in slope_stats['features']]
            avg_slope = mean(means)
        print("Average Slope:", avg_slope)


    ## ---------------------------
    # calculate average rainfall for each storm period

    rainfalls = []

    print('calculating rainfall average for')
    for rr in rainfall_rasters:

        avg_rainfall = None

        #table_rainfall_avg = self.so("rainfall_avg", p, "fgdb")
        table_rainfall_avg = CreateUniqueName(
            "basin{0}_rainfall_avg{1}".format(point.uid, rr['freq']),
            "in_memory"
        )

        with EnvManager(
            cellSizeProjectionMethod="PRESERVE_RESOLUTION",
            extent="MINOF",
            cellSize=one_shed
        ):
            ZonalStatisticsAsTable(
                one_shed,
                'Value',
                Raster(str(rr['path'])),
                table_rainfall_avg,
                "DATA",
                "MEAN"
            )

        rainfall_stats = json.loads(RecordSet(table_rainfall_avg).JSON)

        if len(rainfall_stats['features']) > 0:
            means = [f['attributes']['MEAN'] for f in rainfall_stats['features']]
            avg_rainfall = mean(means) * 25.4 / 1000

        print(rr['freq'], "year event:", avg_rainfall)
        rainfalls.append(Rainfall(rr['freq'], '24hr', avg_rainfall))

    shed = Shed(
        uid=point.uid,
        area_sqkm=total_area,
        avg_slope_pct=avg_slope,
        avg_cn=avg_cn,
        max_fl=fl_max,
        rainfall=rainfalls,
        shed_polygon_filepath = cpd,
        shed_raster_filepath = one_shed_path            
    )
    point.shed = shed
    sheds.append(shed)

--------------------------------
inlet 70978
deriving FeatureSet
[26918]
<geoprocessing spatial reference object object at 0x0000016FBA5FEE70>
[Point(uid=70978, lat=4690423.416399388, lng=618918.2765349224, spatial_ref_code=26918, group_id=None, naacc=None, capacity=None, shed=None, analytics=[], include=True, validation_errors={}, notes=[], raw={'OBJECTID': 1, 'pointid': 1, 'grid_code': 70978, 'Survey_ID': 73607, 'NAACC_ID': 70978, 'Lat': 42.35717, 'Long': -73.556041, 'Rd_Name': 'Raup Road', 'Culv_Mat': 'Metal', 'In_Type': 'Projecting', 'In_Shape': 'Round', 'In_A': 4.9, 'In_B': 4.8, 'HW': 2, 'Slope': 2.8, 'Length': 26, 'Out_Shape': 'Round Culvert', 'Out_A': 5.2, 'Out_B': 5.4, 'Crossing_T': 'Culvert', 'Comments': ' ', 'Flags': 1, 'Modeling_n': ' ', 'BarrierID': '96C19', 'BarrierID_': 0, 'BarrierID1': 0, 'geometry': {'x': 618918.2765349224, 'y': 4690423.416399388}})]
<geoprocessing record set object object at 0x0000016FBA721F50> {"displayFieldName":"","fieldAliases":{"OID":"OID","uid":"

in_memory\catchment_71049.tif <class 'arcpy.sa.Raster.Raster'>
in_memory\basin71049
area: 229475.304935253
flow length: 1077.1088867188
Average Curve Number: 73.64634146341463
Average Slope: 4.215719690695664
calculating rainfall average for
1 year event: 56.67912648083623
2 year event: 69.89079930313588
5 year event: 91.46438222996515
10 year event: 109.35403588850173
25 year event: 133.97999965156797
50 year event: 152.24901602787457
100 year event: 171.9700362369338
200 year event: 195.73032020905922
500 year event: 231.7626982578397
1000 year event: 262.51408885017423
--------------------------------
inlet 70962
deriving FeatureSet
[26918]
<geoprocessing spatial reference object object at 0x0000016FBA721770>
[Point(uid=70962, lat=4689973.416399388, lng=615548.2765349215, spatial_ref_code=26918, group_id=None, naacc=None, capacity=None, shed=None, analytics=[], include=True, validation_errors={}, notes=[], raw={'OBJECTID': 6, 'pointid': 6, 'grid_code': 70962, 'Survey_ID': 73590, 'NA

in_memory\catchment_72494.tif <class 'arcpy.sa.Raster.Raster'>
in_memory\basin72494
area: 34781.104565461676
flow length: 281.34494018555
Average Curve Number: 58.35632183908046
Average Slope: 1.5085030776159516
calculating rainfall average for
1 year event: 57.658
2 year event: 71.29780000000001
5 year event: 93.599
10 year event: 112.141
25 year event: 137.61719999999997
50 year event: 156.4894
100 year event: 176.911
200 year event: 201.5744
500 year event: 239.0648
1000 year event: 271.1196
--------------------------------
inlet 70995
deriving FeatureSet
[26918]
<geoprocessing spatial reference object object at 0x0000016FBA727590>
[Point(uid=70995, lat=4689743.416399388, lng=620528.2765349206, spatial_ref_code=26918, group_id=None, naacc=None, capacity=None, shed=None, analytics=[], include=True, validation_errors={}, notes=[], raw={'OBJECTID': 10, 'pointid': 10, 'grid_code': 70995, 'Survey_ID': 73624, 'NAACC_ID': 70995, 'Lat': 42.350532, 'Long': -73.53654, 'Rd_Name': 'Ten Broek Ro

In [16]:
print(points[0])

Point(uid=70978, lat=4690423.416399388, lng=618918.2765349224, spatial_ref_code=26918, group_id=None, naacc=None, capacity=None, shed=Shed(uid=70978, group_id=None, area_sqkm=3198.260236505569, avg_slope_pct=16.102196872234344, avg_cn=77.75, max_fl=88.260292053223, rainfall=[Rainfall(freq=1, dur='24hr', value=59.74079999999999), Rainfall(freq=2, dur='24hr', value=74.1172), Rainfall(freq=5, dur='24hr', value=97.6122), Rainfall(freq=10, dur='24hr', value=117.1194), Rainfall(freq=25, dur='24hr', value=143.9672), Rainfall(freq=50, dur='24hr', value=163.80459999999997), Rainfall(freq=100, dur='24hr', value=185.3438), Rainfall(freq=200, dur='24hr', value=211.63279999999997), Rainfall(freq=500, dur='24hr', value=251.94259999999997), Rainfall(freq=1000, dur='24hr', value=286.61359999999996)], inlet_geom=None, shed_geom=None, shed_polygon_filepath='in_memory\\basin70978_dissolved', shed_raster_filepath='in_memory/catchment_70978.tif'), analytics=[], include=True, validation_errors={}, notes=[],

In [43]:
PointSchema = class_schema(Point)

In [44]:
test_point = PointSchema().dump(points[0])

In [45]:
pp(test_point)

{ 'analytics': [],
  'capacity': None,
  'group_id': None,
  'include': True,
  'lat': 4690423.416399388,
  'lng': 618918.2765349224,
  'naacc': None,
  'notes': [],
  'raw': { 'BarrierID': '96C19',
           'BarrierID1': 0,
           'BarrierID_': 0,
           'Comments': ' ',
           'Crossing_T': 'Culvert',
           'Culv_Mat': 'Metal',
           'Flags': 1,
           'HW': 2,
           'In_A': 4.9,
           'In_B': 4.8,
           'In_Shape': 'Round',
           'In_Type': 'Projecting',
           'Lat': 42.35717,
           'Length': 26,
           'Long': -73.556041,
           'Modeling_n': ' ',
           'NAACC_ID': 70978,
           'OBJECTID': 1,
           'Out_A': 5.2,
           'Out_B': 5.4,
           'Out_Shape': 'Round Culvert',
           'Rd_Name': 'Raup Road',
           'Slope': 2.8,
           'Survey_ID': 73607,
           'geometry': {'x': 618918.2765349224, 'y': 4690423.416399388},
           'grid_code': 70978,
           'pointid': 1},
  'shed'

In [48]:
json.dumps(test_point)

'{"shed": {"area_sqkm": 3198.260236505569, "avg_slope_pct": 16.102196872234344, "rainfall": [{"freq": 1, "value": 59.74079999999999, "dur": "24hr"}, {"freq": 2, "value": 74.1172, "dur": "24hr"}, {"freq": 5, "value": 97.6122, "dur": "24hr"}, {"freq": 10, "value": 117.1194, "dur": "24hr"}, {"freq": 25, "value": 143.9672, "dur": "24hr"}, {"freq": 50, "value": 163.80459999999997, "dur": "24hr"}, {"freq": 100, "value": 185.3438, "dur": "24hr"}, {"freq": 200, "value": 211.63279999999997, "dur": "24hr"}, {"freq": 500, "value": 251.94259999999997, "dur": "24hr"}, {"freq": 1000, "value": 286.61359999999996, "dur": "24hr"}], "shed_raster_filepath": "in_memory/catchment_70978.tif", "avg_cn": 77.75, "max_fl": 88.260292053223, "uid": "70978", "shed_geom": null, "shed_polygon_filepath": "in_memory\\\\basin70978_dissolved", "inlet_geom": null, "group_id": null}, "analytics": [], "capacity": null, "spatial_ref_code": 26918, "lng": 618918.2765349224, "include": true, "validation_errors": {}, "notes": [

In [49]:
all_points_dicts = []
for p in points:
    all_points_dicts.append(PointSchema().dump(p))

In [51]:
with open(r"C:\Users\chris\dev\drainage\drainit\tests\point_after_delin.json", 'w') as fp:
    json.dump({"points": all_points_dicts}, fp)